In [1]:

#Prompts for fake detector
#_____________________________________________________________________

LIST_OF_CHANGES = [
    ["Name of casualty or group", "Name of casualty or group is the name of the casualty or the name of the group associated with the casualty.", """Name of casualty or group is the name of the casualty or the name of the group associated with the casualty. Is the "Name of casualty or group" in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""],
    ["Gender or age group", "Gender or age group indicates if the casualty is male or female, or specifies their age group.", """Gender or age group indicates if the casualty is male or female, or specifies their age group. Is the "Gender or age group" of the casualty in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""],
    ["Cause of death", "Cause of death is the weapon used in the attack (e.g., shooting, shelling, chemical weapons, etc.).", """Cause of death is the weapon used in the attack (e.g., shooting, shelling, chemical weapons, etc.). Is the "Cause of death" in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""],
    ["Type", "Type is the information if the casualty is civilian or non-civilian.", """Type is the information if the casualty is civilian or non-civilian. Is the "Type" (civilian or non-civilian) in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""],
    ["Actor", "The actor is the person or group responsible for the attack.", """The actor is the person or group responsible for the attack. Is the "Actor" (group responsible for the attack) in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""],
    ["Place of death", "Place of death refers to the cities or areas where the attacks happened.", """Place of death refers to the cities or areas where the attacks happened. Is the "Place of death" in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""],
    ["Date of death", "The date of death refers to theme where the attack happened in the article.", """The date of death refers to when the attack happened in the article. Is the "Date of death" in the article approximately coherent with this description: {}? If the description comes from the article, output "The answer is true" and otherwise output "The answer is false". In addition to "The answer is true" or "The answer is false" label provide short explanation. Do not be to specific, but consider if the information matches overall."""]
]


prompt = """You will be given an event in Syrian war dated from 2013 to 2017. Pleas read and understand the event that is stored in JSON format:

{events}

You must check that the event presented in the article is from among previously red events. Try to check that all the information matches. That means that "Name of casualty or group" "Gender or age group", "Cause of death", "Type", "Actor", "Place of death" and "Date of death" must match.

{article}

If the article matches some event print 'true', else print 'false'. In addition to 'true' or 'false' provide explanation.
"""

prompt_one_by_one = """Carefully read through the article and try to understand its {topic}. {meaning_of_topic}

{article}

{question}
"""

#Imports
import random
import ollama
import json
import re
import os
import pandas as pd

#Suplementary functions

def find_json(text):
        """Function to find JSON-like content in the text."""
        content = []
        pattern0 = (
            r'.*Name of casualty or group.*:\s*(.*),.*\n'
            r'.*Gender or age group.*:\s*(.*),.*\n'
            r'.*Cause of death.*:\s*(.*),.*\n'
            r'.*Type.*:\s*(.*),.*\n'
            r'.*Actor.*:\s*(.*),.*\n'
            r'.*Place of death.*:\s*(.*),.*\n'
            r'.*Date of death.*:\s*(.*)\s*\n'
        )
        pattern1 = (
            r'\{.*Name of casualty or group.*:\s*(.*),.*'
            r'.*Gender or age group.*:\s*(.*),.*'
            r'.*Cause of death.*:\s*(.*),.*'
            r'.*Type.*:\s*(.*),.*'
            r'.*Actor.*:\s*(.*),.*'
            r'.*Place of death.*:\s*(.*),.*'
            r'.*Date of death.*:\s*(.*)\}.*'
        )
        
        
        pattern2 = r'(?:"Name of casualty or group": \[([^\]]*?)\],?\n\s*"Gender or age group": \[([^\]]*?)\],?\n\s*"Cause of death": \[([^\]]*?)\],?\n\s*"Type": \[([^\]]*?)\],?\n\s*"Actor": \[([^\]]*?)\],?\n\s*"Place of death": \[([^\]]*?)\],?\n\s*"Date of death": \[([^\]]*?)\])'

        matches = re.finditer(pattern2, text)

        for match in matches:
            content_dict = {
                "Name of casualty or group": match.group(1).replace('\"', ''),
                "Gender or age group": match.group(2).replace('\"', ''),
                "Cause of death": match.group(3).replace('\"', ''),
                "Type": match.group(4).replace('\"', ''),
                "Actor": match.group(5).replace('\"', ''),
                "Place of death": match.group(6).replace('\"', ''),
                "Date of death": match.group(7).replace('\"', ''),
            }
            content.append(content_dict)

        
        matches = re.finditer(pattern0, text)

        for match in matches:
            content_dict = {
                "Name of casualty or group": match.group(1).replace('\"', ''),
                "Gender or age group": match.group(2).replace('\"', ''),
                "Cause of death": match.group(3).replace('\"', ''),
                "Type": match.group(4).replace('\"', ''),
                "Actor": match.group(5).replace('\"', ''),
                "Place of death": match.group(6).replace('\"', ''),
                "Date of death": match.group(7).replace('\"', ''),
            }
            content.append(content_dict)

        matches = re.finditer(pattern1, text)

        for match in matches:
            content_dict = {
                "Name of casualty or group": match.group(1).replace('\"', ''),
                "Gender or age group": match.group(2).replace('\"', ''),
                "Cause of death": match.group(3).replace('\"', ''),
                "Type": match.group(4).replace('\"', ''),
                "Actor": match.group(5).replace('\"', ''),
                "Place of death": match.group(6).replace('\"', ''),
                "Date of death": match.group(7).replace('\"', ''),
            }
            content.append(content_dict)

        return content

def find_first_true_or_false(text):
    """Finds the first occurrence of the words 'true' or 'false' in the text and returns it with its position."""
    # Regular expression pattern to match 'true' or 'false'
    pattern = r'\b(The answer is true|The answer is false|The answer is True|The answer is False|The answer is TRUE|The answer is FALSE|the answer is true|the answer is false|the answer is True|the answer is False|the answer is TRUE|the answer is FALSE)\b'

    # Search for the first occurrence
    match = re.search(pattern, text, re.IGNORECASE)  # Using IGNORECASE to match 'True', 'False' etc.

    if match:
        word = match.group(1)  # Get the matched word ('true' or 'false')
        return word.lower()
    else:
        return None # Return None if not found, and -1 for position
    

#______________________________________________________________________________________-
#main function

def fake_detect_comparison_true_to_true_and_false_to_true(list_of_changed_articles, model_name = "llama3.1:8b", print_comments = False):   
    data = []
    for i, list_of_data in enumerate(list_of_changed_articles):
        num_of_false = 0
        num_of_true = 0
        information = []

        if print_comments:
            print("Article: ", list_of_data[3])
            print("Fasts: \n", list_of_data[4])

        for  topic, meaning_of_topic, question in LIST_OF_CHANGES:
            response = ollama.chat(model=model_name, messages=[
                    {
                        'role': 'user',
                        'content': prompt_one_by_one.format(article = list_of_data[3], meaning_of_topic = meaning_of_topic, question = question.format(find_json(list_of_data[4])[0][topic]), topic = topic),
                        'temperature':0.2,
                    },
                ])

            generated = response['message']['content']
            if print_comments == True:
                print("This was generated when comparing topic: ", topic, " for the true article.")
                print(generated)

            opinion = find_first_true_or_false(generated)
            if opinion == "the answer is true":
                num_of_true += 1
            elif opinion == "the answer is false":
                num_of_false += 1
            else:
                pass
        print('True article:')
        if num_of_false + num_of_true == len(LIST_OF_CHANGES):
            print(num_of_false)
            print(data)
            first_information = [num_of_false, num_of_false + num_of_true]
        else:
            print(num_of_false, "Missing!!!")
            first_information = [num_of_false, num_of_false + num_of_true]

        num_of_false = 0
        num_of_true = 0

        if print_comments:
            print("Article: ", list_of_data[1])
            print("Fasts: \n", list_of_data[4])

        for  topic, meaning_of_topic, question in LIST_OF_CHANGES:
            response = ollama.chat(model=model_name, messages=[
                    {
                        'role': 'user',
                        'content': prompt_one_by_one.format(article = list_of_data[1], meaning_of_topic = meaning_of_topic, question = question.format(find_json(list_of_data[4])[0][topic]), topic = topic),
                        'temperature':0.2,
                    },
                ])

            generated = response['message']['content']

            if print_comments == True:
                print("This was generated when comparing topic: ", topic, ", with the false article.")
                print(generated)
    
            opinion = find_first_true_or_false(generated)
            if opinion == "the answer is true":
                num_of_true += 1
            elif opinion == "the answer is false":
                num_of_false += 1
            else:
                pass
        print('False article:')
        if num_of_false + num_of_true == len(LIST_OF_CHANGES):
            print(num_of_false)
            data.append([first_information[0], first_information[1], num_of_false, num_of_false + num_of_true])
        else:
            print(num_of_false, "Missing!!!")
            data.append([first_information[0], first_information[1], num_of_false, num_of_false + num_of_true])
    
    return data


import pandas as pd
import os


load_folder = "changed_articles"
name_of_loaded_file = "fake_articles_one_by_one_generation.csv"
path_string_load = os.path.join(load_folder, name_of_loaded_file)
model_name = "mixtral:8x7b"

data = pd.read_csv(path_string_load)
data = data.dropna()

print(data.values.tolist())
print(data.shape)


results = fake_detect_comparison_true_to_true_and_false_to_true(data.values.tolist(), model_name, print_comments = True)


save_folder = "saved_results"
name_of_saved_file = "results_53_data_clean_mixtral:8x7b.csv"
path_save_string = os.path.join(save_folder, name_of_saved_file)



d:\Users\mihac\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


[[34, "A devastating bomb blast in Aleppo has left six people dead and over 30 injured. The attack occurred in the southwestern neighborhood of Salaheddin, a former front-line area between opposition and regime-held sectors before Bashar al-Assad's forces regained control of the city.\n\nEvacuations of civilians from conflict zones have resumed after a four-day pause. Convoy buses carrying thousands of people left besieged towns for regime-controlled areas, but a previous day's convoy attack had already claimed 126 lives, including over 60 children. The UK-based Syrian Observatory for Human Rights reported that the convoy was carrying at least 5000 people.\n\nThe previous attack forced a population exchange deal brokered by Iran and Qatar to be halted. As evacuations resume, residents of besieged towns remain in a precarious situation, vulnerable to further attacks.", "{'Name of casualty or group': 'Syrians', 'Gender or age group': 'children, adults (civilians), men (pro-regime fighter

In [2]:
import pandas as pd
import os


load_folder = "changed_articles"
name_of_loaded_file = "fake_articles_one_by_one_generation.csv"
path_string_load = os.path.join(load_folder, name_of_loaded_file)
model_name = "gemma2:9b"

data = pd.read_csv(path_string_load)
data = data.dropna()

print(data.values.tolist())
print(data.shape)

save_folder = "saved_prints"
name_of_saved_file = "results_53_data_clean_mixtral:8x7b_prits.txt"
path_string_save_comments = os.path.join(save_folder, name_of_saved_file)

results2 = fake_detect_comparison_true_to_true_and_false_to_true(data.values.tolist(), model_name, print_comments = True)


save_folder = "saved_results"
name_of_saved_file = "results_53_data_clean_gemma2.csv"
path_save_string = os.path.join(save_folder, name_of_saved_file)

columns = ["False for true article", "All responses for true article", "False for false article", "All responses for false article"]
results2 = pd.DataFrame(results2, columns=columns)

results2.to_csv(path_save_string, index = False)


[[34, "A devastating bomb blast in Aleppo has left six people dead and over 30 injured. The attack occurred in the southwestern neighborhood of Salaheddin, a former front-line area between opposition and regime-held sectors before Bashar al-Assad's forces regained control of the city.\n\nEvacuations of civilians from conflict zones have resumed after a four-day pause. Convoy buses carrying thousands of people left besieged towns for regime-controlled areas, but a previous day's convoy attack had already claimed 126 lives, including over 60 children. The UK-based Syrian Observatory for Human Rights reported that the convoy was carrying at least 5000 people.\n\nThe previous attack forced a population exchange deal brokered by Iran and Qatar to be halted. As evacuations resume, residents of besieged towns remain in a precarious situation, vulnerable to further attacks.", "{'Name of casualty or group': 'Syrians', 'Gender or age group': 'children, adults (civilians), men (pro-regime fighter

In [14]:
save_folder = "saved_results"
name_of_saved_file = "results_53_data_clean_mixtral8x7b.csv"
path_save_string = os.path.join(save_folder, name_of_saved_file)


results3 = [[1, 7, 4, 7], [1, 7, 2, 7], [0, 7, 1, 7], [1, 7, 1, 7], [0, 7, 1, 7], [1, 7, 2, 6], [1, 7, 2, 7], [1, 7, 2, 6], [1, 7, 3, 7], [1, 7, 3, 7], [0, 7, 1, 7], [1, 7, 1, 6], [0, 7, 1, 7], [1, 7, 1, 7], [1, 7, 2, 7], [0, 7, 0, 7], [2, 7, 4, 6], [1, 7, 0, 7], [1, 7, 2, 7], [0, 7, 0, 7], [0, 7, 1, 7], [1, 7, 1, 7], [4, 7, 5, 7], [3, 7, 1, 4], [1, 7, 2, 7], [1, 7, 1, 7], [2, 7, 2, 7], [0, 7, 1, 7], [3, 7, 4, 7], [3, 7, 4, 7], [1, 7, 1, 7], [0, 7, 1, 6], [0, 7, 0, 6], [1, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [1, 7, 2, 6], [0, 7, 1, 7], [1, 7, 2, 7], [0, 7, 2, 7], [1, 7, 1, 7], [1, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [2, 7, 3, 6], [0, 7, 1, 7], [0, 7, 1, 7], [1, 7, 2, 7]]

columns = ["False for true article", "All responses for true article", "False for false article", "All responses for false article"]
results3 = pd.DataFrame(results3, columns=columns)

results3.to_csv(path_save_string, index = False)



In [15]:

save_folder = "saved_results"
name_of_saved_file = "results_53_data_clean_gemma2.csv"
path_save_string = os.path.join(save_folder, name_of_saved_file)

results2 = [[2, 7, 2, 7], [0, 7, 0, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 3, 7], [1, 7, 2, 7], [1, 7, 2, 7], [0, 7, 3, 7], [1, 7, 1, 7], [0, 7, 1, 7], [0, 7, 0, 7], [0, 7, 2, 7], [1, 7, 2, 7], [0, 7, 1, 7], [0, 7, 0, 7], [0, 7, 3, 7], [0, 7, 0, 7], [2, 7, 3, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [3, 7, 5, 7], [2, 7, 0, 7], [1, 7, 2, 7], [1, 7, 1, 7], [1, 7, 3, 7], [0, 7, 1, 7], [1, 7, 0, 7], [1, 7, 0, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 2, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 1, 7], [0, 7, 3, 7], [0, 7, 1, 7], [1, 7, 2, 7], [1, 7, 3, 7], [1, 7, 1, 7], [0, 7, 2, 7], [0, 7, 1, 7], [0, 7, 2, 7], [0, 7, 1, 7], [1, 7, 2, 7], [0, 7, 0, 7], [0, 7, 1, 7], [0, 7, 1, 7]]
columns = ["False for true article", "All responses for true article", "False for false article", "All responses for false article"]

results2 = pd.DataFrame(results2, columns=columns)
results2.to_csv(path_save_string, index = False)
